In [9]:
import pandas as pd

articles = pd.read_csv('validated_articles.csv')

print(f"Validated articles: {len(articles)}")

articles.head(1)

Validated articles: 2014


,authors,title,date,year,edition,url,text,quarter,year_quarter
0,NaN,本期导读,2023-01-01,2023,1.0,http://www.qstheory.cn/dukan/qs/2023-01/01/c_1...,本期发表了习近平总书记的重要讲话《为实现党的二十大确定的目标任务而团结奋斗》。讲话强调，全面...,1,2023 Q1


In [10]:
# preprocessing
import re
import jieba 

def remove_punctuation(text):
    try:
        return re.sub('[，。：；‘“’”！？（） 【】「」/、｜《》]', "", text)
    except:
        return ""

# remove punctuation
articles['cleaned_text'] = articles['text'].map(remove_punctuation)
articles.head(1)

,authors,title,date,year,edition,url,text,quarter,year_quarter,cleaned_text
0,NaN,本期导读,2023-01-01,2023,1.0,http://www.qstheory.cn/dukan/qs/2023-01/01/c_1...,本期发表了习近平总书记的重要讲话《为实现党的二十大确定的目标任务而团结奋斗》。讲话强调，全面...,1,2023 Q1,本期发表了习近平总书记的重要讲话为实现党的二十大确定的目标任务而团结奋斗讲话强调全面贯彻党的...


In [11]:
# parse the Chinese words to have spaces
def parse_chinese(text):
    seg_text = jieba.cut(text)
    seg_list = [word for word in seg_text]
    return " ".join(seg_list)

articles['cleaned_text_wspaces'] = articles["cleaned_text"].map(lambda x: parse_chinese(x))

articles.head(1)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/r4/qqn8tv4d7g18t_5fsxt6y1340000gn/T/jieba.cache
Loading model cost 0.321 seconds.
Prefix dict has been built successfully.


,authors,title,date,year,edition,url,text,quarter,year_quarter,cleaned_text,cleaned_text_wspaces
0,NaN,本期导读,2023-01-01,2023,1.0,http://www.qstheory.cn/dukan/qs/2023-01/01/c_1...,本期发表了习近平总书记的重要讲话《为实现党的二十大确定的目标任务而团结奋斗》。讲话强调，全面...,1,2023 Q1,本期发表了习近平总书记的重要讲话为实现党的二十大确定的目标任务而团结奋斗讲话强调全面贯彻党的...,本期 发表 了 习近平 总书记 的 重要讲话 为 实现 党 的 二十大 确定 的 目标 任务...


In [12]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/calebharding/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [13]:
def get_ngrams(text, n):
    n_grams = ngrams(word_tokenize(text), n)
    return [ ''.join(grams) for grams in n_grams]

articles['unigram'] = articles['cleaned_text_wspaces'].map(lambda x: get_ngrams(x, 1))

In [14]:
articles['bigram'] = articles['cleaned_text_wspaces'].map(lambda x: get_ngrams(x, 2))

In [15]:
articles.head(1)

,authors,title,date,year,edition,url,text,quarter,year_quarter,cleaned_text,cleaned_text_wspaces,unigram,bigram
0,NaN,本期导读,2023-01-01,2023,1.0,http://www.qstheory.cn/dukan/qs/2023-01/01/c_1...,本期发表了习近平总书记的重要讲话《为实现党的二十大确定的目标任务而团结奋斗》。讲话强调，全面...,1,2023 Q1,本期发表了习近平总书记的重要讲话为实现党的二十大确定的目标任务而团结奋斗讲话强调全面贯彻党的...,本期 发表 了 习近平 总书记 的 重要讲话 为 实现 党 的 二十大 确定 的 目标 任务...,"[本期, 发表, 了, 习近平, 总书记, 的, 重要讲话, 为, 实现, 党, 的, 二十...","[本期发表, 发表了, 了习近平, 习近平总书记, 总书记的, 的重要讲话, 重要讲话为, ..."


In [16]:
# create unigram count dictionary
def ngram_counts(ngram):
    # create word count dictionary
    word_count = {}

    for word in ngram:
        if word in word_count.keys():
            cur_val = word_count[word]
            cur_val += 1
            word_count[word] = cur_val
        else:
            word_count[word] = 1

    # sort the dictionary
    word_count = {k: v for k, v in sorted(word_count.items(), key=lambda item: item[1], reverse=True)}

    return word_count

articles['unigram_count'] = articles["unigram"].map(lambda x: ngram_counts(x))
articles['bigram_count'] = articles["bigram"].map(lambda x: ngram_counts(x))

articles.head(1)

,authors,title,date,year,edition,url,text,quarter,year_quarter,cleaned_text,cleaned_text_wspaces,unigram,bigram,unigram_count,bigram_count
0,NaN,本期导读,2023-01-01,2023,1.0,http://www.qstheory.cn/dukan/qs/2023-01/01/c_1...,本期发表了习近平总书记的重要讲话《为实现党的二十大确定的目标任务而团结奋斗》。讲话强调，全面...,1,2023 Q1,本期发表了习近平总书记的重要讲话为实现党的二十大确定的目标任务而团结奋斗讲话强调全面贯彻党的...,本期 发表 了 习近平 总书记 的 重要讲话 为 实现 党 的 二十大 确定 的 目标 任务...,"[本期, 发表, 了, 习近平, 总书记, 的, 重要讲话, 为, 实现, 党, 的, 二十...","[本期发表, 发表了, 了习近平, 习近平总书记, 总书记的, 的重要讲话, 重要讲话为, ...","{'的': 31, '新': 12, '发展': 9, '党': 8, '文章': 8, '...","{'党的': 8, '的文章': 6, '的二十大': 5, '新的': 4, '文章阐释'..."


In [17]:
articles.to_csv('processed_articles.csv', index=False)